In [1]:
pip install langchain langgraph langchain_openai langchainhub langsmith duckduckgo-search beautifulsoup4 gradio

  Using cached langchain_openai-0.1.8-py3-none-any.whl.metadata (2.5 kB)
  Using cached duckduckgo_search-6.1.6-py3-none-any.whl.metadata (19 kB)
  Using cached gradio-4.36.1-py3-none-any.whl.metadata (15 kB)
INFO: pip is looking at multiple versions of langgraph to determine which version is compatible with other requirements. This could take a while.
  Using cached langgraph-0.0.66-py3-none-any.whl.metadata (23 kB)
  Using cached openai-1.34.0-py3-none-any.whl.metadata (21 kB)
  Using cached pyreqwest_impersonate-0.4.7-cp311-none-win_amd64.whl.metadata (9.8 kB)
  Using cached orjson-3.10.4-cp311-none-win_amd64.whl.metadata (50 kB)
  Using cached ffmpy-0.3.2-py3-none-any.whl
  Using cached gradio_client-1.0.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.7.4-py3-none-any.whl.metadata (109 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached ruff-0.4.8-py3-none-win_amd64.whl.metadata (24 kB)
  Using cached semantic_version-2.10.0-py2.py3-

  You can safely remove it manually.


In [2]:
pip install getpass4


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-any.whl size=1860 sha256=bd10b0da637ee910ba2f6ceb86b233f81032eae4306f64e752ffdc2f34448eb8
  Stored in directory: c:\users\shrey\appdata\local\pip\cache\wheels\20\0a\a4\f32deed892cc142168e64d59e26c7906fb7b6fcdb0e3bfb71f
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=29c0e8265c4c166fbb0b87fd7d41864d99a52dc83226a6dd9b1dc150bac3a63a
  Stored in directory: c:\users\shrey\appdata\local\pip\cache\wheels\70\bd\ba\8ae5c080c895c9360fe6e153acda2dee82527374467eae061b
Successfully built clipboard pyperclip


In [3]:
import getpass
open_ai_api_key=getpass.getpass("Enter your open ai api key")

Enter your open ai api key········


In [28]:
import functools, operator, requests, os, json
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import gradio as gr
# Set environment variables
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Research Agents"



# Initialize model

llm = ChatOpenAI(model="gpt-4-turbo-preview",api_key=open_ai_api_key)

# Define custom tools
@tool("internet_search", return_direct=False)
def internet_search(query: str) -> str:
    """Searches the internet using DuckDuckGo."""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=5)]
        return results if results else "No results found."

@tool("process_content", return_direct=False)
def process_content(url: str) -> str:
    """Processes content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

tools = [internet_search, process_content]

# Helper function for creating agents
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

# Define agent nodes
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

# Create Agent Supervisor
members = ["Web_Searcher", "Insight_Researcher","Summariser"]
system_prompt = (
    "As a supervisor, your role is to oversee a dialogue between the workers and perform  task as defined below by choosing the appropiate workesr  accordingly"
    " workers: {members}."
    "Task : Based on the input you need to search the web for the topic using relevant search terms , if any insights are provided , make sure to follow them as well."
    "After getting the result based on the seacrh , you need to do an insight research based on the result."
    " Then, in the end you need to make a condensed summary of the content from previous steps, return the summary it as a result"
    " Once all steps are complete,"
    " indicate with 'FINISH'."
)

options = ["FINISH"] + members
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}] }},
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
]).partial(options=str(options), members=", ".join(members))

supervisor_chain = (prompt | llm.bind_functions(functions=[function_def], function_call="route") | JsonOutputFunctionsParser())

# Define the Agent State and Graph
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

search_agent = create_agent(llm, tools, "You are a web searcher. Search the internet for information.")
search_node = functools.partial(agent_node, agent=search_agent, name="Web_Searcher")

insights_research_agent = create_agent(llm, tools, 
        """You are a Insight Researcher. Do step by step. 
        Based on the provided content first identify the list of relevant sub topics based on the main topic,
        then search internet for each topic one by one
        and finally find insights for each topic one by one.
        Include only the insights  in the final response
        """)
insights_research_node = functools.partial(agent_node, agent=insights_research_agent, name="Insight_Researcher")


summary_agent = create_agent(llm, tools, "You are an expert content summariser and have a lot of professional experience in the media. Given the content make a condensed summary of the content relevant to the topic. Make sure the summary is worthy of reading and is interesting and engaging and can be posted as a content on any social site.")
summary_node = functools.partial(agent_node, agent=summary_agent, name="Summariser")

workflow = StateGraph(AgentState)
workflow.add_node("Web_Searcher", search_node)
workflow.add_node("Insight_Researcher", insights_research_node)
workflow.add_node("Summariser", summary_node)
workflow.add_node("supervisor", supervisor_chain)


# Define edges
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

graph = workflow.compile()




# Run the graph
def run_graph(title,user_insight):
    input_message=title+" "+user_insight
    response = graph.invoke({
        "messages": [HumanMessage(content=input_message)]
    })
    #print(json.dumps(response['messages'][1].content, indent=2))
    print(response)
    return json.dumps(response['messages'][-1].content, indent=2)
    #summary_markdown = response['messages'][1].content
    #summary_text = markdown_to_text(summary_markdown)
    #print(summary_text)
    #return summary_text

#outputs = gr.Textbox(label="outputs")

demo = gr.Interface(fn=run_graph, inputs=["text","text"], outputs=gr.components.Markdown())
demo.launch()


········
Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


{'messages': [HumanMessage(content='the ousting of elon musk from paypal and its aftermath '), HumanMessage(content="Elon Musk's ousting from PayPal in 2000 marked a pivotal moment in his career, leading to significant developments in his entrepreneurial journey. This event was orchestrated by key PayPal members, including Peter Thiel, Max Levchin, Reid Hoffman, and David Sacks. Musk had merged his company, X.com, with Confinity, which owned PayPal, and disagreements over the company's branding and strategy, coupled with Musk's management style, led to his removal.\n\nWhile Musk was on a honeymoon in Australia, Thiel and Levchin engineered his ouster as CEO, a decision supported by other board members like Hoffman. Musk was initially angry about his removal, contemplating thoughts of retribution. However, he later recognized the ousting as beneficial, allowing him to focus on future endeavors beyond PayPal. Despite being ousted, Musk maintained his equity stake in PayPal, which proved 

In [39]:

import functools
import operator
import requests
import os
import json
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence, TypedDict
import gradio as gr
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser



# Set environment variables
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Research Agents"

# Initialize model
llm = ChatOpenAI(model="gpt-4-turbo-preview", api_key=open_ai_api_key)

# Initialize Chroma vector store
embeddings = OpenAIEmbeddings(api_key=open_ai_api_key)
vector_store = Chroma(embedding_function=embeddings)

# Define custom tools
@tool("internet_search", return_direct=False)
def internet_search(query: str) -> str:
    """Searches the internet using DuckDuckGo."""
    with DDGS() as ddgs:
        results = [r['body'] for r in ddgs.text(query, max_results=3)]
        return results if results else "No results found."

@tool("process_content", return_direct=False)
def process_content(url: str) -> str:
    """Processes content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

tools = [internet_search, process_content]

# Helper function for creating agents
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

# Define agent nodes
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # Store intermediate result in vector store
    text = result["output"]
    vector_store.add_texts([text])
    return {"messages": [HumanMessage(content=text, name=name)]}

# Create Agent Supervisor
members = ["Web_Searcher", "Insight_Researcher", "Summariser"]
system_prompt = (
    "As a supervisor, your role is to oversee a dialogue between the workers and perform tasks as defined below by choosing the appropriate workers accordingly. "
    "Workers: {members}. "
    "Task: Based on the input, you need to search the web for the topic using relevant search terms. "
    "After getting the result based on the search, you need to do insight research based on the result. "
    "Once all steps are complete, indicate with 'FINISH'."
)

options = ["FINISH"] + members
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}] }},
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
]).partial(options=str(options), members=", ".join(members))

supervisor_chain = (prompt | llm.bind_functions(functions=[function_def], function_call="route") | JsonOutputFunctionsParser())

# Define the Agent State and Graph
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

search_agent = create_agent(llm, tools, "You are a web searcher. Search the internet for information.")
search_node = functools.partial(agent_node, agent=search_agent, name="Web_Searcher")

insights_research_agent = create_agent(llm, tools, 
        """You are an Insight Researcher. Do step by step. 
        Based on the provided content, first identify the list of relevant subtopics based on the main topic,
        then search the internet for each topic one by one,
        and finally find insights for each topic one by one.
        Include only the insights in the final response.
        """)
insights_research_node = functools.partial(agent_node, agent=insights_research_agent, name="Insight_Researcher")

summary_agent = create_agent(llm, tools, "You are an expert content summarizer and have a lot of professional experience in the media. Given the content, make a condensed summary of the content relevant to the topic. Make sure the summary is worthy of reading and is interesting and engaging and can be posted as content on any social site.")
summary_node = functools.partial(agent_node, agent=summary_agent, name="Summariser")

workflow = StateGraph(AgentState)
workflow.add_node("Web_Searcher", search_node)
workflow.add_node("Insight_Researcher", insights_research_node)
workflow.add_node("Summariser", summary_node)
workflow.add_node("supervisor", supervisor_chain)

# Define edges
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

graph = workflow.compile()

def generate_summarization_prompt(texts):
    prompt = (
        "You are an expert content summarizer. Given the following texts, create a condensed, "
        "information-dense summary that connects all the subtopics. The summary should be engaging and suitable for posting as content on any social site.\n\n"
        "Texts:\n" + "\n\n".join(texts) + "\n\nSummary:"
    )
    return prompt

# Run the graph
def run_graph(title, user_insight):
    input_message = title + " " + user_insight
    response = graph.invoke({
        "messages": [HumanMessage(content=input_message)]
    })

    # Retrieve relevant information from vector store for summarization
    texts = [doc.page_content for doc in vector_store.similarity_search(input_message, k=10)]

    # Generate the final summary
    summarization_prompt = generate_summarization_prompt(texts)

    # Generate the final summary
    summary_response = llm.invoke(summarization_prompt)
    parser = StrOutputParser()
    summary_parse_response=parser.invoke(summary_response)
   
    print(summary_parse_response)
    return summary_parse_response

outputs = gr.Textbox(label="outputs")
demo = gr.Interface(fn=run_graph, inputs=["text", "text"], outputs=outputs)
demo.launch()


········
Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


Elon Musk's departure from PayPal in 2000, following a pivotal merger with Confinity, was a watershed moment that not only altered the course of his career but also had a lasting impact on the tech landscape. Despite the challenges posed by his ousting, Musk's entrepreneurial spirit remained undeterred, propelling him into ventures that would revolutionize industries and underscore his innovative prowess. This event marked the beginning of an astounding trajectory for Musk, leading to the founding of SpaceX and Tesla, and later contributions to Neuralink and The Boring Company. Interestingly, the so-called "PayPal Mafia," a group of former PayPal employees who went on to make significant marks on the tech industry, played a dual role in Musk's narrative—as catalysts of his PayPal exit and as crucial supporters of his endeavors, notably during SpaceX's critical moments. This narrative arc from rivalry to support encapsulates the complex dynamics of Silicon Valley and highlights the esse

In [41]:
query = "ousting of elon musk from paypal and aftermath"
results = vector_store.similarity_search(query, k=3)

# Print results
for result in results:
    print(result)

page_content='The ousting of Elon Musk from PayPal in 2000, following the merger of his company X.com with Confinity, marked a significant turn in his career. Led by investor Peter Thiel and supported by PayPal\'s board, this decision was based on concerns over Musk\'s vision for the company. This event catalyzed the rise of the "PayPal Mafia," a group of former PayPal members who later made significant contributions to the tech industry, including aiding Musk\'s SpaceX during financial struggles.\n\nPost-PayPal, Musk embarked on ventures that would revolutionize industries, notably founding Tesla Motors and SpaceX, showcasing his resilience and innovative spirit. His departure also led to PayPal\'s growth, culminating in its acquisition by eBay in 2002, highlighting Musk\'s lasting impact on the company\'s success. This narrative showcases the transformative journey of Elon Musk from a ousted CEO to a pivotal figure in technology and space exploration.'
page_content='The story of Elon